## Install Depedencies

In [1]:
!pip install -q transformers subword-nmt attacut tokenizers numpy==1.21.6 gdown torchmetrics==0.7

In [2]:
!git clone https://github.com/lstnlp/hoogberta
%cd hoogberta
!pip install -q torchtext==0.6.0 fairseq==0.10.2 pytorch-lightning==1.4.7 textsearch seqeval
!pip install -q --no-dependencies --editable  .

Cloning into 'hoogberta'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 267 (delta 63), reused 73 (delta 43), pack-reused 161
Receiving objects: 100% (267/267), 4.05 MiB | 21.81 MiB/s, done.
Resolving deltas: 100% (136/136), done.
/kaggle/working/hoogberta


In [3]:
from hoogberta import download
download()

Downloading...
From: https://drive.google.com/uc?id=1xQHDAE8nbFu2wAM6SAXtTjk890JWLUhy
To: /kaggle/working/hoogberta/dict.zip
100%|██████████| 1.15k/1.15k [00:00<00:00, 2.54MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1bBSWQzzEt99mYd_EY5W-lQKW6L-D8axW
From (redirected): https://drive.google.com/uc?id=1bBSWQzzEt99mYd_EY5W-lQKW6L-D8axW&confirm=t&uuid=d52406ec-36f8-4bb2-a592-bff702324407
To: /kaggle/working/hoogberta/modelL12.pt
100%|██████████| 575M/575M [00:02<00:00, 265MB/s]  
Downloading...
From (uriginal): https://drive.google.com/uc?id=1fYtRAyh6d4W9LVCSJiSYKKM_CCPflBc9
From (redirected): https://drive.google.com/uc?id=1fYtRAyh6d4W9LVCSJiSYKKM_CCPflBc9&confirm=t&uuid=91a0483e-de82-41b9-a03e-6782ff233066
To: /kaggle/working/hoogberta/checkpoint_best.pt
100%|██████████| 1.44G/1.44G [00:06<00:00, 207MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZNxpVHNZbAfdWA-wu7iMSUtcySzQCJam
To: /kaggle/working/hoogberta/dict.txt
100%|██████████| 1.09M/1.09M [00:00<

## Convert Model

In [4]:
%cd /kaggle/working/hoogberta

/kaggle/working/hoogberta


In [5]:
base_path = '/kaggle/working/hoogberta'

In [6]:
from hoogberta.multitagger import HoogBERTaMuliTaskTagger
tagger = HoogBERTaMuliTaskTagger(cuda=False) # or cuda=True
output = tagger.nlp("วันที่ 12 มีนาคมนี้ ฉันจะไปเที่ยววัดพระแก้ว ที่กรุงเทพ")

In [7]:
import torch
hoogberta_base_checkpoint = torch.load('/kaggle/working/hoogberta/models/hoogberta_base/checkpoint_best.pt', map_location=torch.device('cpu'))

In [8]:
hoogberta_base_checkpoint['model'] = tagger.model.encoder.model.state_dict()
torch.save(hoogberta_base_checkpoint, '/kaggle/working/hoogberta/models/hoogberta_base/model.pt')

In [9]:
from transformers.models.roberta.convert_roberta_original_pytorch_checkpoint_to_pytorch import convert_roberta_checkpoint_to_pytorch

convert_roberta_checkpoint_to_pytorch('models/hoogberta_base', 'converted_model_l12', False)

1042301B [00:00, 49378519.83B/s]
456318B [00:00, 35098778.89B/s]


Our BERT config: RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 74905
}



Configuration saved in converted_model_l12/config.json


torch.Size([1, 11, 74905]) torch.Size([1, 11, 74905])
max_absolute_diff = 1.9073486328125e-06
Do both models output the same tensors? 🔥
Saving model to converted_model_l12


Model weights saved in converted_model_l12/pytorch_model.bin


In [10]:
from transformers import AutoTokenizer, RobertaForTokenClassification, RobertaModel
from attacut import tokenize
import torch

tokenizer = AutoTokenizer.from_pretrained("new5558/HoogBERTa")

huggingface_model_classification = RobertaForTokenClassification.from_pretrained('converted_model_l12')

loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--new5558--HoogBERTa/snapshots/0fadd23185e3ba91012653a71962c280fe97fa67/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--new5558--HoogBERTa/snapshots/0fadd23185e3ba91012653a71962c280fe97fa67/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--new5558--HoogBERTa/snapshots/0fadd23185e3ba91012653a71962c280fe97fa67/tokenizer_config.json
loading configuration file converted_model_l12/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position

In [11]:
huggingface_model_classification.classifier = tagger.model.fc_ne

In [12]:
sentence = "วันที่ 12 มีนาคมนี้ ฉันจะไปเที่ยววัดพระแก้ว ที่กรุงเทพ"

huggingface_model_classification.classifier = tagger.model.fc_ne.eval()
tagger.model.eval()
all_sent = []
sentences = sentence.split(" ")
for sent in sentences:
    all_sent.append(" ".join(tokenize(sent)).replace("_","[!und:]"))

sentence = " _ ".join(all_sent)
tokenized_text = tokenizer(sentence, return_tensors = 'pt')
token_ids = tokenized_text['input_ids']

with torch.no_grad():
    ne_pred = huggingface_model_classification(**tokenized_text).logits
    ne_out  =  ne_pred.argmax(dim = -1).view(-1).tolist()
    ne   = [tagger.ne_dict[id] for id in ne_out]

In [13]:
ne

['<s>',
 'B_DTM',
 'I_DTM',
 'I_DTM',
 'I_DTM',
 'I_DTM',
 'E_DTM',
 'E_DTM',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B_LOC',
 'O',
 'O',
 'B_LOC',
 '</s>']

In [14]:
output

[['วัน', 'NN', 'B_DTM', 'O'],
 ['ที่', 'PS', 'I_DTM', 'O'],
 [' ', 'PU', 'I_DTM', 'PUNC'],
 ['12', 'NU', 'I_DTM', 'O'],
 [' ', 'PU', 'I_DTM', 'PUNC'],
 ['มีนาคม', 'NN', 'E_DTM', 'O'],
 ['นี้', 'AJ', 'E_DTM', 'O'],
 [' ', 'PU', 'O', 'MARK'],
 ['ฉัน', 'PR', 'O', 'O'],
 ['จะ', 'AX', 'O', 'O'],
 ['ไป', 'AV', 'O', 'O'],
 ['เที่ยว', 'VV', 'O', 'O'],
 ['วัดพระแก้ว', 'NN', 'B_LOC', 'O'],
 [' ', 'PU', 'O', 'PUNC'],
 ['ที่', 'PS', 'O', 'O'],
 ['กรุงเทพ', 'NN', 'B_LOC', 'O']]

In [15]:
id2label = {}
label2id = {}
for i in range(len(tagger.ne_dict.symbols)):
    id2label[i] = tagger.ne_dict.symbols[i]
    label2id[tagger.ne_dict.symbols[i]] = i

In [16]:
huggingface_model_classification.config.id2label = id2label
huggingface_model_classification.config.label2id = label2id

In [17]:
from transformers import pipeline

sentence = "วันที่ 12 มีนาคมนี้ ฉันจะไปเที่ยววัดพระแก้ว ที่กรุงเทพ"

all_sent = []
sentences = sentence.split(" ")
for sent in sentences:
    all_sent.append(" ".join(tokenize(sent)).replace("_","[!und:]"))

sentence = " _ ".join(all_sent)

nlp = pipeline('token-classification', model=huggingface_model_classification, tokenizer=tokenizer, aggregation_strategy="none")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
nlp(sentence)

[{'entity': 'B_DTM',
  'score': 0.98959374,
  'index': 1,
  'word': 'วัน</w>',
  'start': 0,
  'end': 3},
 {'entity': 'I_DTM',
  'score': 0.9782606,
  'index': 2,
  'word': 'ที่</w>',
  'start': 4,
  'end': 7},
 {'entity': 'I_DTM',
  'score': 0.9975866,
  'index': 3,
  'word': '_</w>',
  'start': 8,
  'end': 9},
 {'entity': 'I_DTM',
  'score': 0.9989825,
  'index': 4,
  'word': '12</w>',
  'start': 10,
  'end': 12},
 {'entity': 'I_DTM',
  'score': 0.9988531,
  'index': 5,
  'word': '_</w>',
  'start': 13,
  'end': 14},
 {'entity': 'E_DTM',
  'score': 0.8283873,
  'index': 6,
  'word': 'มีนาคม</w>',
  'start': 15,
  'end': 21},
 {'entity': 'E_DTM',
  'score': 0.5152181,
  'index': 7,
  'word': 'นี้</w>',
  'start': 22,
  'end': 25},
 {'entity': 'B_LOC',
  'score': 0.97150415,
  'index': 13,
  'word': 'วัดพระแก้ว</w>',
  'start': 45,
  'end': 55},
 {'entity': 'B_LOC',
  'score': 0.93494374,
  'index': 16,
  'word': 'กรุงเทพ</w>',
  'start': 62,
  'end': 69}]

In [19]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("new5558/HoogBERTa")

loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--new5558--HoogBERTa/snapshots/0fadd23185e3ba91012653a71962c280fe97fa67/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--new5558--HoogBERTa/snapshots/0fadd23185e3ba91012653a71962c280fe97fa67/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--new5558--HoogBERTa/snapshots/0fadd23185e3ba91012653a71962c280fe97fa67/tokenizer_config.json


In [20]:
tokenizer.save_pretrained('saved_tokenizer')

tokenizer config file saved in saved_tokenizer/tokenizer_config.json
Special tokens file saved in saved_tokenizer/special_tokens_map.json


('saved_tokenizer/tokenizer_config.json',
 'saved_tokenizer/special_tokens_map.json',
 'saved_tokenizer/tokenizer.json')

In [21]:
tokenizer_loaded = PreTrainedTokenizerFast(tokenizer_file = "saved_tokenizer/tokenizer.json")

In [22]:
# huggingface_model_classification.push_to_hub()
# tokenizer_loaded.push_to_hub()